In [1]:
import cv2
import numpy as np
import pandas as pd
import time
from scipy.spatial.distance import squareform, pdist
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from queue import Queue

In [2]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [3]:
video='1.mp4'

# video='/home/admin-pc/Downloads/test7.mp4'

# video='/home/admin-pc/Downloads/abhi_drill.mp4'

imsize = 600
font = cv2.FONT_HERSHEY_SIMPLEX 
  
# org 
org = (100, 100) 
  
# fontScale 
fontScale = 1
   
# Blue color in BGR 
color = (255, 255, 255) 
  
# Line thickness of 2 px 
thickness = 2

# result1 = cv2.VideoWriter('first.mp4', cv2.VideoWriter_fourcc(*'MJPG'), 20,(600,600))
# result2 = cv2.VideoWriter('second.mp4', cv2.VideoWriter_fourcc(*'MJPG'), 20,(600,600))

In [4]:
cap=cv2.VideoCapture(video)
backSub = cv2.createBackgroundSubtractorMOG2(50,16,False)
backSub_slow = cv2.createBackgroundSubtractorMOG2(50,16,False)
# kernel = np.ones((2,2),np.uint8)
kernel=cv2.getStructuringElement(cv2.MORPH_CROSS,(2,2))
kernel_close=cv2.getStructuringElement(cv2.MORPH_CROSS,(2,2))
#backSub2=cv2.bgsegm.createBackgroundSubtractorGMG(); 

In [5]:
def scintillating_pixels(image):
    u,v=np.where(image!=0)
    
    arr=np.array([u,v]).T
    return arr,arr.shape[0]
    
def get_dist_matrix(df_array):
    #df_array = df.to_numpy()
    dist_mat = np.sqrt(cdist(df_array, df_array))
    return dist_mat

def convert_pixels(cord_arr,image):
    for cord  in cord_arr:
        image[cord[0],cord[1]]=0
    return image

In [6]:
x_thresh=2
y_thresh=2

def pixel_count_along_axes(image):
    im_new=image.copy()
    im_new[im_new!=0]=1
    #im_new=image
    
    return im_new.sum(axis=0),im_new.sum(axis=1)



def thresholding(avg_sumx,avg_sumy):
    l=[]
    
    lx=[avg_sumx>x_thresh]
    ly=[avg_sumy>y_thresh]

    #for x in avg_sumx:
        #print(x)
        #lx=[]
        #for y in avg_sumy:
            #if x>x_thresh and y>y_thresh:
                #lx.append(True)
            #else:
                #lx.append(False)
                
        #if sum(lx)>20:
            #print(len(lx))
        #l.append(lx)
                
    return lx,ly
    
def remove_minor_perturbation(image,sumx_avg,sumy_avg):
    image_dum=image.copy()
    if max(sumy_avg)>=2*max(sumx_avg):
        image_dum[image_dum!=0]=0

    if max(sumy_avg)<20 and max(sumx_avg)<20:
        image_dum[image_dum!=0]=0
    return image_dum



def window_thresholding(image,lx):
    im_dum = image.copy()
    #print(lx)
    ids = np.where(lx[0]==False)[0]
    #print(ids)
    #f = lambda x:im_dum[:,x:x-1+window_size]*0
    
    
    for idx in ids: 
        im_dum[:,idx:idx-1+window_size]=0
    return im_dum
            
            

# def window_thresholding(image,lst,avg_sumx,avg_sumy):
#     done=[]
#     im_dum=image.copy()
#     for i in range(len(avg_sumx)):
#         for idx_y,ls in enumerate(lst):
#             if idx_y not in done:
#                 if sum(ls)>20:
#                     if avg_sumx[i]>x_thresh and ls==False:
#                         im_dum[i:i-1+window_size,idx_y:idx_y-1+window_size]=0
#                         done.append(idx_y)
#     return im_dum
                


def motion_estimation(frm1,frm5):
    frame1=frm1.copy()
    frame5=frm5.copy()
    diff=cv2.absdiff(frame1,frame5)
    return diff
   
    
def frm_averaging(frame_buffer):
    cv2.accumulateWeighted(f,avg1,0.1)
    res1 = cv2.convertScaleAbs(avg1)

In [26]:
frm_count=0
count_after=0
dist_mat=0
window_size=30
frame_buffer=[]
ret,frame=cap.read()
avg1 = np.float32(cv2.resize(frame,(imsize,imsize)))
# frame_buffer = Queue(maxsize = 10)
#opening_convert=np.zeros((640, 540))
while True:
    ret,frame=cap.read()
    frame=cv2.resize(frame,(imsize,imsize))
    if frm_count%1==0:
        if ret:
            gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


            fgMask = backSub.apply(gray_img, learningRate=0.002)
            fgMask_slow = backSub_slow.apply(gray_img,learningRate=0.0002)
            #fgMask2 = backSub2.apply(gray_img)
            
            fgmask = cv2.erode(fgMask,kernel,iterations = 1)
            opening = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)
            opening_orig=opening.copy()
            
            fgMask_slow = cv2.erode(fgMask_slow,kernel,iterations = 1)
            opening_slow = cv2.morphologyEx(fgMask_slow, cv2.MORPH_OPEN, kernel)
            #opening_slow = cv2.morphologyEx(opening_slow, cv2.MORPH_CLOSE, kernel_close)
            
            
            #sumx=
            #sumy=
            cord_arr,count=scintillating_pixels(opening)
            sumx,sumy=pixel_count_along_axes(opening)
            sumx_avg=moving_average(sumx,window_size)
            sumy_avg=moving_average(sumy,window_size)
            opening=remove_minor_perturbation(opening,sumx_avg,sumy_avg)
            frame_buffer.append(opening)
            
            #if frm_count%5==0:
            try:
                motion_frames=motion_estimation(frame_buffer[-3],frame_buffer[-1])
            except:
                pass

            if len(frame_buffer)==20:
                frame_buffer.pop(0)
                
            #print(max(sumx_avg),max(sumy_avg))
            lx,ly=thresholding(sumx_avg,sumy_avg)
            #print(lx,ly)
            #opening=window_thresholding(opening,lx)
            
            
            
            cord_arr,count=scintillating_pixels(opening_slow)
            sumx,sumy=pixel_count_along_axes(opening_slow)
            sumx_avg=moving_average(sumx,window_size)
            sumy_avg=moving_average(sumy,window_size)
            opening_slow=remove_minor_perturbation(opening_slow,sumx_avg,sumy_avg)
#             #print(max(sumx_avg),max(sumy_avg))
            lx,ly=thresholding(sumx_avg,sumy_avg)
#             #print(lx,ly)
            if frm_count==0:
                avg1 = np.float32(cv2.resize(opening_slow,(imsize,imsize)))
            
            #opening_slow=window_thresholding(opening_slow,lx)
            cv2.accumulateWeighted(opening_slow,avg1,0.07)
            res1 = cv2.convertScaleAbs(avg1)
            res1[res1<250]=0
            #print(lst)
            #if frm_count!=0:
                #try:
                    #dist_mat=get_dist_matrix(cord_arr)
                    #print(sum(dist_mat).sum(),np.var(dist_mat),count)
                    #if sum(dist_mat).sum()>45808 and count>150:

                       # opening_convert=convert_pixels(cord_arr,opening)

                        #cord_arr,count_after=scintillating_pixels(opening_convert)
                        #cv2.putText(opening_convert, str(np.var(dist_mat)), (150,150), font,  fontScale, color, thickness, cv2.LINE_AA)
                        #cv2.putText(opening_convert, '{} and {}'.format(count, count_after), org, font, fontScale, color, thickness, cv2.LINE_AA)

                #except Exception as e:
                    #pass


            #print(opening.shape)


            #cv2.imshow("Original Image", cv2.resize(opening_convert, (640, 540)) )

            #cv2.imshow("Original Image",frame)
            #cv2.imshow("Original Image2", fgMask2)
            cv2.putText(opening_slow, '{} and {}'.format(round(max(sumx_avg)),round(max(sumy_avg))), org, font, fontScale, color, thickness, cv2.LINE_AA)
            cv2.imshow("Image", frame)
            #cv2.imshow("Image2", opening_slow)
            try:
                cv2.imshow("residual", res1)
                cv2.imshow("frame_diffrence", cv2.absdiff(motion_frames,opening_slow))
            except:
                pass
            #result1.write(opening_orig)
            #result2.write(opening)
            if cv2.waitKey(20)& 0xFF == ord('q'):
                break
            #time.sleep(0.05)
        else:
            print('corrupt!')
    frm_count+=1
cv2.destroyAllWindows()

error: OpenCV(3.4.9) /io/opencv/modules/imgproc/src/resize.cpp:4045: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [8]:
cv2.destroyAllWindows()

In [9]:
np.unique(res1[res1<250])

array([0], dtype=uint8)

In [10]:
frm_count=0
count_after=0
dist_mat=0
window_size=30
while True:
    ret,frame=cap.read()
    frame=cv2.resize(frame,(imsize,imsize))
    if frm_count%1==0:
        if ret:
            gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


            fgMask = backSub.apply(gray_img, learningRate=0.02)
            fgMask_slow = backSub.apply(gray_img,learningRate=0.0002)
            
            cv2.imshow("Image", fgMask)
            cv2.imshow("Image2", fgMask_slow)
            #result1.write(opening_orig)
            #result2.write(opening)
            
            if cv2.waitKey(50)& 0xFF == ord('q'):
                break
cv2.destroyAllWindows()

In [11]:
window_size=30

In [12]:
def my_thresh(img):
    opening=img.copy()
    cord_arr,count=scintillating_pixels(opening)
    sumx,sumy=pixel_count_along_axes(opening)
    sumx_avg=moving_average(sumx,window_size)
    sumy_avg=moving_average(sumy,window_size)
    result=remove_minor_perturbation(opening,sumx_avg,sumy_avg)
    return result

In [45]:
c = cv2.VideoCapture(video)
_,f = c.read()

avg1 = np.float32(cv2.resize(f,(imsize,imsize)))
avg2 = np.float32(cv2.resize(f,(imsize,imsize)))
avg3 = np.float32(cv2.resize(f,(imsize,imsize)))

while(1):
    _,f = c.read()
    f=cv2.resize(f,(imsize,imsize))
    frm1=cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)
    
    cv2.accumulateWeighted(f,avg1,0.01)
    cv2.accumulateWeighted(f,avg2,0.0001)

    res1 = cv2.convertScaleAbs(avg1)
    res2 = cv2.convertScaleAbs(avg2)

    cv2.imshow('img',f)
    cv2.imshow('avg1',res1)
    cv2.imshow('avg2',res2)
    
    gry1=cv2.cvtColor(res1, cv2.COLOR_BGR2GRAY)
    gry2=cv2.cvtColor(res2, cv2.COLOR_BGR2GRAY)
    
    fgMaskfrm = backSub.apply(frm1,learningRate=-1)
    opening_frm = cv2.morphologyEx(fgMaskfrm, cv2.MORPH_OPEN, kernel)
    opening_frm=my_thresh(opening_frm)
   
    fgMask1 = backSub.apply(gry1,learningRate=-1)
    opening1 = cv2.morphologyEx(fgMask1, cv2.MORPH_OPEN, kernel)
    opening2=my_thresh(opening1)
    
    fgMask2 = backSub.apply(gry2,learningRate=-1)
    opening2 = cv2.morphologyEx(fgMask2, cv2.MORPH_OPEN, kernel)
    opening2=my_thresh(opening2)
    
    
    change=cv2.absdiff(cv2.absdiff(res2,res1),cv2.subtract(res2,res1))
    
    cv2.accumulateWeighted(change,avg3,0.0001)

    res3 = cv2.convertScaleAbs(avg3)
  
    cv2.imshow('res3',res3)
       
    #cv2.imshow('diff',cv2.absdiff(res2,res1))
    cv2.imshow('diff2',cv2.absdiff(cv2.absdiff(res2,res1),cv2.subtract(res2,res1)))
    #cv2.imshow('change',cv2.addWeighted(f,0.4,cv2.absdiff(cv2.absdiff(res2,res1),cv2.subtract(res2,res1)),0.1,0))
    if cv2.waitKey(50)& 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
c.release()

error: OpenCV(3.4.9) /io/opencv/modules/imgproc/src/resize.cpp:4045: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [41]:
cv2.destroyAllWindows()

In [35]:
cap = cv2.VideoCapture(video)
# Loop untill the end of the video 
while (cap.isOpened()): 
    # Capture frame-by-frame 
    ret, frame = cap.read() 
    #frame = cv2.resize(frame, ((540, 380))#, fx = 0, fy = 0,interpolation = cv2.INTER_CUBIC) 
  
    # Display the resulting frame 
    cv2.imshow('Frame', frame) 
      
    # conversion of BGR to grayscale is necessary to apply this operation 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
  
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU) 
  
    # apply NOT operation on image and mask generated by thresholding 
    BIT = cv2.bitwise_and(frame, frame, mask = mask) 
    cv2.imshow('BIT', BIT) 
  
    # define q as the exit button 
    if cv2.waitKey(25) & 0xFF == ord('q'): 
        break
  
# release the video capture object 
cap.release() 
  
# Closes all the windows currently opened. 
cv2.destroyAllWindows() 